In [15]:
# === FIXED analyze_with_cnn-style Predictions for IG + LSB Pipeline ===

In [16]:
import os
import cv2
import numpy as np
import pandas as pd
import random
import string
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from skimage.metrics import structural_similarity as ssim
from scipy.fftpack import dct
import tensorflow as tf
from tensorflow.keras.models import load_model
from tf_keras_vis.saliency import Saliency
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear

In [17]:
# === CONFIGURATION === #
IMG_SIZE = (128, 128)
NUM_PIXELS = 32
BITS_PER_COORD = 16
RESERVED_BITS = NUM_PIXELS * BITS_PER_COORD * 2
MESSAGE_BITS = 4 * 8
CHANNELS = 1
CLASS_NAMES = ['Normal', 'Attack']

In [18]:
# === MESSAGE UTILS === #
def generate_random_message():
    return ''.join(random.choices(string.ascii_letters + string.digits, k=4))

def string_to_bits(s):
    return [int(b) for c in s.encode('utf-8') for b in format(c, '08b')][:MESSAGE_BITS]

def bits_to_string(bits):
    chars = []
    for i in range(0, len(bits), 8):
        byte = bits[i:i+8]
        if len(byte) == 8:
            chars.append(chr(int(''.join(map(str, byte)), 2)))
    return ''.join(chars)

In [19]:
# === COORDINATE CONVERSION === #
def coords_to_bits(coords):
    bits = []
    for y, x in coords:
        yb = format(y, f'0{BITS_PER_COORD}b')
        xb = format(x, f'0{BITS_PER_COORD}b')
        bits.extend(int(b) for b in yb + xb)
    return bits

def bits_to_coords(bits):
    coords = []
    for i in range(0, len(bits), 32):
        y = int(''.join(map(str, bits[i:i+16])), 2)
        x = int(''.join(map(str, bits[i+16:i+32])), 2)
        coords.append((y, x))
    return coords

In [20]:
# === CLEANING FUNCTION === #
def clean_message(s):
    return ''.join(c for c in s if c.isprintable())

In [21]:
# === EMBEDDING === #
def embed_lsb(image, bits, coords):
    flat = image.flatten()
    for i, (y, x) in enumerate(coords):
        idx = y * image.shape[1] + x
        flat[idx] = (flat[idx] & ~1) | bits[i]
    return flat.reshape(image.shape)

def embed_metadata(image, meta_bits):
    flat = image.flatten()
    for i in range(len(meta_bits)):
        flat[i] = (flat[i] & ~(1 << 7)) | (meta_bits[i] << 7)
    return flat.reshape(image.shape)

In [22]:
# === IG-BASED IMPORTANT PIXELS === #
def get_ig_pixels(model, image_tensor, top_k=NUM_PIXELS):
    def loss_fn(output): return output[:, 0]  # Single output from sigmoid
    saliency = Saliency(model, model_modifier=ReplaceToLinear(), clone=True)
    saliency_map = saliency(loss_fn, image_tensor)
    saliency_map = np.abs(saliency_map[0])
    if saliency_map.ndim == 3:
        saliency_map = np.mean(saliency_map, axis=-1)
    flat = saliency_map.flatten()
    indices = np.argsort(flat)[-top_k:]
    ys, xs = np.unravel_index(indices, saliency_map.shape)
    return list(zip(ys, xs))

In [23]:
def visualize_ig(image, saliency_map, true_class, pred_class, filename, output_dir):
    if saliency_map.ndim == 3:
        saliency_map = np.mean(saliency_map, axis=-1)

    # Normalize saliency map for visualization
    norm_saliency = (saliency_map - saliency_map.min()) / (saliency_map.max() - saliency_map.min() + 1e-8)

    # Plot
    plt.figure(figsize=(18, 6))

    # 1. Original Image
    plt.subplot(1, 3, 1)
    plt.imshow(image, cmap='gray')
    plt.title(f"Original Image\nTrue Class: {true_class} | Predicted Class: {pred_class}")
    plt.axis('off')

    # 2. Scaled Raw IG
    plt.subplot(1, 3, 2)
    plt.imshow(norm_saliency, cmap='hot')
    plt.title("Integrated Gradients (Raw, Scaled)")
    plt.axis('off')
    plt.colorbar(label="Pixel Importance (Scaled Raw IG Values)", shrink=0.7)

    # 3. Overlay
    plt.subplot(1, 3, 3)
    overlay = cv2.cvtColor((image.astype(np.float32) / 255.0), cv2.COLOR_GRAY2RGB)
    plt.imshow(overlay, cmap='gray')
    plt.imshow(norm_saliency, cmap='turbo', alpha=0.5)
    plt.title("Integrated Gradients (Overlay)")
    plt.axis('off')
    plt.colorbar(label="Pixel Importance (Normalized IG Values)", shrink=0.7)

    os.makedirs(output_dir, exist_ok=True)
    out_path = os.path.join(output_dir, f"ig_vis_{filename}.png")
    plt.tight_layout()
    plt.savefig(out_path)
    plt.close()


In [24]:
def visualize_ig(image, saliency_map, true_class, pred_class, filename, output_dir):
    if saliency_map.ndim == 3:
        saliency_map = np.mean(saliency_map, axis=-1)

    # Flatten, get top-100 pixel indices
    flat = saliency_map.flatten()
    top_indices = np.argsort(flat)[-200:]
    mask = np.zeros_like(flat)
    mask[top_indices] = flat[top_indices]
    masked_saliency = mask.reshape(saliency_map.shape)

    # Normalize only non-zero entries
    nonzero = masked_saliency[masked_saliency > 0]
    if nonzero.size > 0:
        min_val = nonzero.min()
        max_val = nonzero.max()
        norm_saliency = (masked_saliency - min_val) / (max_val - min_val + 1e-8)
    else:
        norm_saliency = masked_saliency

    # Prepare color overlay with original grayscale base
    grayscale_rgb = np.stack([image] * 3, axis=-1).astype(np.uint8)

    # Apply 'turbo' colormap to normalized top-100 IG pixels
    cmap = plt.get_cmap('turbo')
    heatmap_rgb = (cmap(norm_saliency)[:, :, :3] * 255).astype(np.uint8)

    # Overlay: keep grayscale, only paint top-100 pixels with color
    overlay = grayscale_rgb.copy()
    top_mask = norm_saliency > 0
    overlay[top_mask] = heatmap_rgb[top_mask]

    # === Plotting ===
    plt.figure(figsize=(18, 6))

    # 1. Original image
    plt.subplot(1, 3, 1)
    plt.imshow(image, cmap='gray')
    plt.title(f"Original Image\nTrue: {true_class} | Pred: {pred_class}")
    plt.axis('off')

    # 2. Raw heatmap (top-100 only)
    plt.subplot(1, 3, 2)
    plt.imshow(norm_saliency, cmap='hot')
    plt.title("Top-100 IG (Raw Scaled)")
    plt.axis('off')
    plt.colorbar(label="Pixel Importance", shrink=0.8)

    # 3. Overlay: grayscale image + sharp colored IG pixels
    plt.subplot(1, 3, 3)
    plt.imshow(overlay)
    plt.title("Top-100 IG Overlay")
    plt.axis('off')
    plt.colorbar(label="Pixel Importance", shrink=0.8)

    os.makedirs(output_dir, exist_ok=True)
    out_path = os.path.join(output_dir, f"ig_vis_{filename}.png")
    plt.tight_layout()
    plt.savefig(out_path)
    plt.close()


In [25]:
# === DECODING === #
def decode_message_and_coords(image):
    flat = image.flatten()
    coord_bits = [(flat[i] >> 7) & 1 for i in range(RESERVED_BITS)]
    coords = bits_to_coords(coord_bits)
    msg_bits = [flat[y * image.shape[1] + x] & 1 for y, x in coords[:NUM_PIXELS]]
    return bits_to_string(msg_bits[:MESSAGE_BITS]), coords

In [26]:
# === QUALITY METRICS === #
def calculate_mse(original, stego):
    return np.mean((original.astype(np.float32) - stego.astype(np.float32)) ** 2)

def calculate_psnr(mse, max_pixel=255.0):
    if mse == 0:
        return float('inf')
    return 20 * math.log10(max_pixel / math.sqrt(mse))

def calculate_ssim(original, stego):
    return ssim(original, stego)

def compute_dct_difference(original, stego):
    original_dct = dct(dct(original.T, norm='ortho').T, norm='ortho')
    stego_dct = dct(dct(stego.T, norm='ortho').T, norm='ortho')
    diff = np.abs(original_dct - stego_dct)
    return np.mean(diff), np.max(diff)

In [27]:
def process_folder(input_folder, model_path, output_excel, stego_output_folder, ig_vis_output_folder):
    model = load_model(model_path)
    results = []
    original_labels = []
    original_preds = []
    stego_preds = []

    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    output_dir = os.path.dirname(output_excel)
    os.makedirs(ig_vis_output_folder, exist_ok=True)

    for idx, filename in enumerate(image_files):
        print(f"[{idx + 1}/{len(image_files)}] Processing {filename}")
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"⚠️ Could not read {filename}. Skipping.")
            continue

        image = cv2.resize(image, IMG_SIZE)
        norm_img = image.astype(np.float32) / 255.0
        input_tensor = np.expand_dims(norm_img, axis=(0, -1))

        # Original prediction
        prob_orig = float(model.predict(input_tensor, verbose=0)[0])
        pred_class_orig = CLASS_NAMES[int(prob_orig >= 0.5)]
        print(f"Original prediction prob: {prob_orig:.4f} → Predicted class: {pred_class_orig}")
        original_preds.append(pred_class_orig)

        # True label from folder name
        folder_name = os.path.basename(os.path.normpath(input_folder)).lower()
        if folder_name == "normal":
            true_label = "Normal"
        elif folder_name == "attack":
            true_label = "Attack"
        else:
            raise ValueError(f"Cannot determine true label from folder name: {folder_name}")
        original_labels.append(true_label)

        # IG pixels
        ig_pixels = get_ig_pixels(model, input_tensor, top_k=NUM_PIXELS)

        # === IG Visualization ===
        saliency = Saliency(model, model_modifier=ReplaceToLinear(), clone=True)
        saliency_map = saliency(lambda output: output[:, 0], input_tensor)
        visualize_ig(
            image=image,
            saliency_map=saliency_map[0],
            true_class=true_label,
            pred_class=pred_class_orig,
            filename=filename.split('.')[0],
            output_dir=ig_vis_output_folder
        )

        # Embed message
        original_msg = generate_random_message()
        msg_bits = string_to_bits(original_msg)
        coord_bits = coords_to_bits(ig_pixels)

        msg_encoded_img = embed_lsb(image.copy(), msg_bits, ig_pixels)
        final_stego = embed_metadata(msg_encoded_img, coord_bits)

        # CNN on stego image
        cnn_input = np.expand_dims(final_stego.astype(np.float32) / 255.0, axis=(0, -1))
        prob_stego = float(model.predict(cnn_input, verbose=0)[0])
        pred_class_after = CLASS_NAMES[int(prob_stego >= 0.5)]
        stego_preds.append(pred_class_after)

        # Decode and evaluate
        decoded_msg, decoded_coords = decode_message_and_coords(final_stego)
        original_clean = clean_message(original_msg)
        decoded_clean = clean_message(decoded_msg)

        print(f"Encoded Message: {original_clean}")
        print(f"Decoded Message: {decoded_clean}")
        print(f"Messages Match: {'Matched' if original_clean == decoded_clean else 'Mismatched'}\n")

        # Metrics
        mse_value = calculate_mse(image, final_stego)
        psnr_value = calculate_psnr(mse_value)
        ssim_value = calculate_ssim(image, final_stego)
        mean_dct_diff, max_dct_diff = compute_dct_difference(image, final_stego)

        # Save stego image
        stego_image_filename = f"stego_{filename}"
        stego_image_path = os.path.join(stego_output_folder, stego_image_filename)
        cv2.imwrite(stego_image_path, final_stego)

        # Collect results
        results.append({
            "Image": filename,
            "Original Message": original_clean,
            "Decoded Message": decoded_clean,
            "Match": "Matched" if original_clean == decoded_clean else "Mismatched",
            "CNN Predicted Class": pred_class_after,
            "Confidence": round(prob_stego if pred_class_after == "Attack" else 1 - prob_stego, 4),
            "Encoded Coords": str(ig_pixels),
            "Decoded Coords Match": "Matched" if ig_pixels == decoded_coords else "Mismatched",
            "True Label": true_label,
            "MSE": round(mse_value, 4),
            "PSNR": round(psnr_value, 2),
            "SSIM": round(ssim_value, 4),
            "Mean DCT Diff": round(mean_dct_diff, 4),
            "Max DCT Diff": round(max_dct_diff, 4)
        })

    # Final outputs
    df = pd.DataFrame(results)
    cm_original = confusion_matrix(original_labels, original_preds, labels=CLASS_NAMES)
    cm_stego = confusion_matrix(original_labels, stego_preds, labels=CLASS_NAMES)

    # Plots
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm_original, annot=True, fmt='d', cmap="Greens", xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Confusion Matrix BEFORE Embedding")
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "confusion_matrix_original.png"))
    plt.close()

    plt.figure(figsize=(6, 5))
    sns.heatmap(cm_stego, annot=True, fmt='d', cmap="Blues", xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Confusion Matrix AFTER Embedding")
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "confusion_matrix_stego.png"))
    plt.close()

    plt.figure(figsize=(6, 4))
    sns.countplot(data=df, x="Match")
    plt.title("Message Match Count")
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "message_match_count.png"))
    plt.close()

    plt.figure(figsize=(6, 4))
    sns.countplot(data=df, x="Decoded Coords Match")
    plt.title("Coordinate Match Count")
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "coordinate_match_count.png"))
    plt.close()

    summary_df = pd.DataFrame({
        "Average MSE": [df["MSE"].mean()],
        "Average PSNR (dB)": [df["PSNR"].mean()],
        "Average SSIM": [df["SSIM"].mean()],
        "Average Mean DCT Diff": [df["Mean DCT Diff"].mean()],
        "Average Max DCT Diff": [df["Max DCT Diff"].mean()],
        "Message Match Rate (%)": [(df["Match"] == "Matched").mean() * 100],
        "Coordinate Match Rate (%)": [(df["Decoded Coords Match"] == "Matched").mean() * 100],
        "CNN Accuracy After Embedding (%)": [(df["True Label"] == df["CNN Predicted Class"]).mean() * 100]
    })

    with pd.ExcelWriter(output_excel, engine='openpyxl', mode='w') as writer:
        df.to_excel(writer, index=False, sheet_name='Per Image Results')
        summary_df.to_excel(writer, index=False, sheet_name='Summary Averages')

    print("\n✅ Done! Results saved to", output_excel)
    print("\n📊 AVERAGE METRICS ACROSS ALL IMAGES:")
    for col in summary_df.columns:
        print(f"{col}: {summary_df[col].iloc[0]:.4f}" if isinstance(summary_df[col].iloc[0], float) else f"{col}: {summary_df[col].iloc[0]}")


In [28]:
# === RUN === #
if __name__ == "__main__":
    input_folder = r"C:\Users\SIL-Laptop\Desktop\Final IAI Gustavo\steganography\Layered IG LSB\CNN FDIA 1\Custom Message encoder\Attack"
    model_path = r"C:\Users\SIL-Laptop\Desktop\Final IAI Gustavo\steganography\Layered IG LSB\CNN FDIA 1\cnn_2_class_model_grayscale.h5"
    output_excel = r"C:\Users\SIL-Laptop\Desktop\Final IAI Gustavo\steganography\Layered IG LSB/CNN FDIA 1/Final Results (Normal)/ig_layered_stego_results.xlsx"
    stego_output_folder = r"C:\Users\SIL-Laptop\Desktop\Final IAI Gustavo\steganography\Layered IG LSB/CNN FDIA 1/Final Results (Normal)/stego_images"
    ig_vis_output_folder = r"C:\Users\SIL-Laptop\Desktop\Final IAI Gustavo\steganography\Layered IG LSB/CNN FDIA 1/ig_visualizations"

    if not os.path.exists(stego_output_folder):
        os.makedirs(stego_output_folder)

    process_folder(input_folder, model_path, output_excel, stego_output_folder, ig_vis_output_folder)

[1/12] Processing confusion_matrix_original (10).png
Original prediction prob: 0.9999 → Predicted class: Attack
Encoded Message: ccGh
Decoded Message: ccGh
Messages Match: Matched

[2/12] Processing confusion_matrix_original (3).png
Original prediction prob: 0.9999 → Predicted class: Attack
Encoded Message: ckTM
Decoded Message: ckTM
Messages Match: Matched

[3/12] Processing confusion_matrix_original (5).png
Original prediction prob: 0.9999 → Predicted class: Attack
Encoded Message: IOPJ
Decoded Message: IOPJ
Messages Match: Matched

[4/12] Processing confusion_matrix_stego (10).png
Original prediction prob: 0.9999 → Predicted class: Attack
Encoded Message: 7oKT
Decoded Message: 7oKT
Messages Match: Matched

[5/12] Processing confusion_matrix_stego (3).png
Original prediction prob: 0.9999 → Predicted class: Attack
Encoded Message: 8gRg
Decoded Message: 8gRg
Messages Match: Matched

[6/12] Processing confusion_matrix_stego (5).png
Original prediction prob: 0.9999 → Predicted class: Att